# Power Curve App

It’ll be served the app with:
```
panel serve app.ipynb --autoreload
```
Now, open the app in your browser at **http://localhost:5006/app**.

### References

- https://coderzcolumn.com/tutorials/data-science/basic-dashboard-using-panel


In [1]:
import os
import hvplot.pandas
import numpy as np
import pandas as pd
import panel as pn
from datetime import datetime, date

PRIMARY_COLOR = "#0072B5"
SECONDARY_COLOR = "#B54300"
FOLDER_DATA= (
    "/Users/juan/Workspace/data/datasets/dashboards"
)

In [ ]:
pn.extension(design="material", sizing_mode="stretch_width")

In [ ]:
#@pn.cache
def load_data(FOLDER_DATA, folder_option):
    folder = os.path.join(FOLDER_DATA, folder_option)
    files = [file for file in os.listdir(folder) if ".csv" in file]
    df = pd.DataFrame()
    for file in files:
        path = os.path.join(folder, file)
        df = pd.concat([df, pd.read_csv(path)], axis = 0)
    return df.dropna()

In [ ]:
def date_to_datetime(dt):
    return datetime(dt.year, dt.month, dt.day, 0, 0, 0)

def transform_data(data):
    data.set_index("Time", inplace = True)
    data.WindSpeed = data.WindSpeed.astype(float) 
    data.Power = data.Power.astype(float) 
    data.index = pd.to_datetime(data.index)
    return data


def get_plot(FOLDER_DATA, folder_option, date_range, date_start, date_end, variable ="Power"):
    data = load_data(FOLDER_DATA, folder_option)
    data = transform_data(data)
    dti = date_to_datetime(np.max([date_range[0], date_start]))   
    dtf = date_to_datetime(np.min([date_range[1], date_end]))
    data = data[(data.index>=dti) & (data.index<=dtf)]
    return data[variable].hvplot(
        height=300, legend=False, color=PRIMARY_COLOR
    )# * highlight.hvplot.scatter(color=SECONDARY_COLOR, padding=0.1, legend=False)


def get_scatter(FOLDER_DATA, folder_option, date_range, date_start, date_end):
    data = load_data(FOLDER_DATA, folder_option)
    data = transform_data(data)
    dti = date_to_datetime(np.max([date_range[0], date_start]))   
    dtf = date_to_datetime(np.min([date_range[1], date_end]))
    data_filtered = data[(data.index>=dti) & (data.index<=dtf)]
    return data.hvplot.scatter(x='WindSpeed', y='Power',
        color=PRIMARY_COLOR, padding=0.1, legend=False, alpha = .5
        )* data_filtered.hvplot.scatter(x='WindSpeed', y='Power', color=SECONDARY_COLOR, padding=0.1, legend=False)


In [ ]:
# collect folder data options
data_options = [folder for folder in os.listdir(FOLDER_DATA) if folder != '.DS_Store']
# widget data option selection
folder_option_widget = pn.widgets.Select(name="Folder Option", value=data_options[0], options=list(data_options))

min_date = date(2014, 10, 1)
max_date = date(2016, 1, 1)
date_range_slider = pn.widgets.DateRangeSlider(
    name='Date Range Slider',
    start=min_date, end=date(2016, 1, 1),
    value=(min_date, date(2016, 1, 1)),
    step=2
)

start_date_picker = pn.widgets.DatePicker(name='Start Date Picker', value=min_date)
end_date_picker = pn.widgets.DatePicker(name='End Date Picker', value=max_date)

In [ ]:
widgets = pn.Column(folder_option_widget, start_date_picker, end_date_picker, sizing_mode="fixed", width=300)
#slider = pn.Row(date_range_slider.controls(jslink=False), date_range_slider)
#slider = pn.Column(date_range_slider)

bound_plot_1 = pn.bind(
    get_plot, FOLDER_DATA=FOLDER_DATA, folder_option=folder_option_widget, date_range = date_range_slider, 
    date_start=start_date_picker, date_end=end_date_picker, variable="WindSpeed"
)
bound_plot_2 = pn.bind(
    get_plot, FOLDER_DATA=FOLDER_DATA, folder_option=folder_option_widget, date_range = date_range_slider, 
    date_start=start_date_picker, date_end=end_date_picker, variable="Power"
)
bound_plot_3 = pn.bind(
    get_scatter, FOLDER_DATA=FOLDER_DATA, folder_option=folder_option_widget, date_range = date_range_slider,
    date_start=start_date_picker, date_end=end_date_picker
)

pn.Column(widgets, date_range_slider, bound_plot_1, bound_plot_2, bound_plot_3)


In [ ]:
pn.template.MaterialTemplate(
    site="Panel",
    title="Prueba App",
    sidebar=[folder_option_widget, start_date_picker, end_date_picker],
    main=[date_range_slider, bound_plot_1, bound_plot_2, bound_plot_3],
).servable();